## Setting Stuff Up 

Here we import some packages that we'll need in various places. This notebooks is not necessary to run if following along with the tutorials in order, or when using the hosted training sandbox. If you are skipping around between notebooks or are recreating your environment with Docker Compose, you can rerun this notebook to bootstrap your environment with the various files, templates, and python modules created 

In [1]:
!mkdir -p ~/agave

%cd ~/agave

!pip -qqq install --upgrade setvar 

import re
import os
import sys
import json
from setvar import *
from time import sleep


/home/jovyan/agave


<h2>Set Variables</h2>

The setvar() package gives us a bash-like way of setting environment variables in Python. It's a lot easier to type `PORT=10022` than it is to type `os.environ["PORT"]="10022"`. The setvar package also supports comments and running a command in the background and getting its output.

In [2]:
setvar("""
AGAVE_APP_NAME=training-${AGAVE_USERNAME}
AGAVE_STORAGE_SYSTEM_ID=sandbox-storage-${AGAVE_USERNAME}
AGAVE_EXECUTION_SYSTEM_ID=sandbox-exec-${AGAVE_USERNAME}
AGAVE_SYSTEM_SITE_DOMAIN=localdomain
MACHINE_NAME=sandbox
MACHINE_USERNAME=jovyan
AGAVE_STORAGE_HOME_DIR=/home/${MACHINE_USERNAME}
SCRATCH_DIR=/home/${MACHINE_USERNAME}
AGAVE_STORAGE_WORK_DIR=/home/${MACHINE_USERNAME}
AGAVE_APP_DEPLOYMENT_PATH=agave-deploy
""")
loadvar()

AGAVE_APP_NAME=training-dooley
AGAVE_STORAGE_SYSTEM_ID=sandbox-storage-dooley
AGAVE_EXECUTION_SYSTEM_ID=sandbox-exec-dooley
AGAVE_SYSTEM_SITE_DOMAIN=localdomain
MACHINE_NAME=sandbox
MACHINE_USERNAME=jovyan
AGAVE_STORAGE_HOME_DIR=/home/jovyan
SCRATCH_DIR=/home/jovyan
AGAVE_STORAGE_WORK_DIR=/home/jovyan
AGAVE_APP_DEPLOYMENT_PATH=agave-deploy
AGAVE_APP_DEPLOYMENT_PATH=agave-deploy
AGAVE_APP_NAME=training-dooley
AGAVE_EXECUTION_SYSTEM_ID=sandbox-exec-dooley
AGAVE_STORAGE_HOME_DIR=/home/jovyan
AGAVE_STORAGE_SYSTEM_ID=sandbox-storage-dooley
AGAVE_STORAGE_WORK_DIR=/home/jovyan
AGAVE_SYSTEM_SITE_DOMAIN=localdomain
MACHINE_NAME=sandbox
MACHINE_USERNAME=jovyan
SCRATCH_DIR=/home/jovyan


If you are running locally using Docker Compose, you will need to pull the ip and port of your reverse tunnel from the sandbox. Uncomment the following command, and enter below

In [25]:
if os.environ.get('USE_TUNNEL') == 'True': 
    # fetch the hostname and port of the reverse tunnel running in the sandbox 
    # so Agave can connect to our local sandbox
    !echo $(ssh -q -o StrictHostKeyChecking=no -o UserKnownHostsFile=/dev/null sandbox 'curl -s  http://localhost:4040/api/tunnels | jq -r '.tunnels[0].public_url'') > ngrok_url.txt  
    !cat ngrok_url.txt | sed 's|^tcp://||' | sed -r 's#(.*):(.*)#\1#' > ngrok_host.txt
    !cat ngrok_url.txt | sed 's|^tcp://||' | sed -r 's#(.*):(.*)#\2#' > ngrok_port.txt

    # set the environment variables otherwise set when running in a training cluster
    os.environ['VM_PORT'] = readfile('ngrok_port.txt').strip()
    os.environ['VM_MACHINE'] = readfile('ngrok_host.txt').strip()
    os.environ['AGAVE_SYSTEM_HOST'] = readfile('ngrok_host.txt').strip()
    os.environ['AGAVE_SYSTEM_PORT'] = readfile('ngrok_port.txt').strip()
    !echo "VM_PORT=$VM_PORT"
    !echo "VM_MACHINE=$VM_MACHINE"
    setvar("VM_IPADDRESS=$(getent hosts ${VM_MACHINE}|cut -d' ' -f1)")

Reading file `ngrok_port.txt'
Reading file `ngrok_host.txt'
Reading file `ngrok_host.txt'
Reading file `ngrok_port.txt'
VM_PORT=15477
VM_MACHINE=0.tcp.ngrok.io
VM_IPADDRESS=52.15.62.13


<h2>PushBullet</h2>
If you wish to use pushbullet to receive status updates, then run the cell below and enter your pushbullet token.

In [4]:
readpass("PBTOK")

Password or secret: PBTOK
········
Writing file `PBTOK.txt'


<h2>Agave Password</h2>

We'll be running several commands that require the Agave password. We'll store it here for convenience.

In [5]:
if not os.environ['AGAVE_PASSWORD']:
    readpass("AGAVE_PASSWORD")

<h2>Setting up Agave</h2>
Agave uses machines called "tenants" to manage user login and authentication. In this step, we tell Agave we are using the standard proction tenant.

In [6]:
!tenants-init

You are now configured to interact with the APIs at https://sandbox.agaveplatform.org/


<h3>Create the Client</h3>

In this next step we delete the client if it exists. Chances are, yours doesn't yet. We put this command here in case, for some reason, you want to re-create your client later on. If you delete the client you intend to create before you create it, no harm is done.

In [8]:
!clients-delete -u $AGAVE_USERNAME -p "$AGAVE_PASSWORD" $AGAVE_APP_NAME

Successfully deleted client training-dooley


In this step we create the client. Clients provide a way of encapsulating resources connected to a single project. Through the client, you will receive a token which you can use to run most of the Agave commands.

In [9]:
!clients-create  -u $AGAVE_USERNAME -p "$AGAVE_PASSWORD" -N $AGAVE_APP_NAME -S

Successfully created client training-dooley
key: Bm1zbK9aBj3XtoYlFez8n4zDLvQa 
secret: Rf7jyli46vjfcVt9mvDXowEuADwa


Create the token for your client. You will, from this point on, use this token to run the remainder of the Agave commands in this tutorial.

In [10]:
!auth-tokens-create -u $AGAVE_USERNAME -p "$AGAVE_PASSWORD"

Token for sandbox:dooley successfully refreshed and cached for 14400 seconds
1a28bc772781f89897779f63207e3036


## Setup Systems & Apps

In [26]:
!mkdir -p ~/key
!chmod 700 ~/key
!jsonpki --public ~/.ssh/id_rsa.pub > ~/key/id_rsa.pub.txt
!jsonpki --private ~/.ssh/id_rsa > ~/key/id_rsa.txt
os.environ["PUB_KEY"]=readfile("${HOME}/key/id_rsa.pub.txt").strip()
os.environ["PRIV_KEY"]=readfile("${HOME}/key/id_rsa.txt").strip()

Reading file `/home/jovyan/key/id_rsa.pub.txt'
Reading file `/home/jovyan/key/id_rsa.txt'


### Storage system

In [28]:
writefile("${AGAVE_STORAGE_SYSTEM_ID}.txt","""{
    "id": "${AGAVE_STORAGE_SYSTEM_ID}",
    "name": "${MACHINE_NAME} storage (${MACHINE_USERNAME})",
    "description": "The ${MACHINE_NAME} computer",
    "site": "${AGAVE_SYSTEM_SITE_DOMAIN}",
    "type": "STORAGE",
    "storage": {
        "host": "${AGAVE_SYSTEM_HOST}",
        "port": ${AGAVE_SYSTEM_PORT},
        "protocol": "SFTP",
        "rootDir": "/",
        "homeDir": "${AGAVE_STORAGE_HOME_DIR}",
        "auth": {
          "username" : "${MACHINE_USERNAME}",
          "publicKey" : "${PUB_KEY}",
          "privateKey" : "${PRIV_KEY}",
          "type" : "SSHKEYS"
        }
    }
}
""")
!systems-addupdate -v -F ${AGAVE_STORAGE_SYSTEM_ID}.txt

Writing file `sandbox-storage-dooley.txt'
{
  "owner": "dooley",
  "_links": {
    "owner": {
      "href": "https://sandbox.agaveplatform.org/profiles/v2/dooley"
    },
    "metadata": {
      "href": "https://sandbox.agaveplatform.org/meta/v2/data/?q=%7B%22associationIds%22%3A%228871979044684033560-242ac11b-0001-006%22%7D"
    },
    "credentials": {
      "href": "https://sandbox.agaveplatform.org/systems/v2/sandbox-storage-dooley/credentials"
    },
    "roles": {
      "href": "https://sandbox.agaveplatform.org/systems/v2/sandbox-storage-dooley/roles"
    },
    "self": {
      "href": "https://sandbox.agaveplatform.org/systems/v2/sandbox-storage-dooley"
    }
  },
  "available": true,
  "description": "The sandbox computer",
  "storage": {
    "proxy": null,
    "protocol": "SFTP",
    "mirror": false,
    "port": 15477,
    "auth": {
      "type": "SSHKEYS"
    },
    "publicAppsDir": null,
    "host": "0.tcp.ngrok.io",
    "rootDir": "/",
    "homeDir": "/home/jovyan"
  },
  "t

### Execution System

In [32]:
# Edit any parts of this file that you know need to be changed for your machine.
writefile("${AGAVE_EXECUTION_SYSTEM_ID}.txt","""
{
    "id": "${AGAVE_EXECUTION_SYSTEM_ID}",
    "name": "${MACHINE_NAME} (${MACHINE_USERNAME})",
    "description": "The ${MACHINE_NAME} computer",
    "site": "${AGAVE_SYSTEM_SITE_DOMAIN}",
    "public": false,
    "status": "UP",
    "type": "EXECUTION",
    "executionType": "CLI",
    "scheduler" : "FORK",
    "environment": null,
    "scratchDir" : "${SCRATCH_DIR}",
    "queues": [
        {
            "name": "none",
            "default": true,
            "maxJobs": 10,
            "maxUserJobs": 10,
            "maxNodes": 6,
            "maxProcessorsPerNode": 6,
            "minProcessorsPerNode": 1,
            "maxRequestedTime": "00:30:00"
        }
    ],
    "login": {
        "auth": {
          "username" : "${MACHINE_USERNAME}",
          "publicKey" : "${PUB_KEY}",
          "privateKey" : "${PRIV_KEY}",
          "type" : "SSHKEYS"
        },
        "host": "${AGAVE_SYSTEM_HOST}",
        "port": ${AGAVE_SYSTEM_PORT},
        "protocol": "SSH"
    },
    "maxSystemJobs": 50,
    "maxSystemJobsPerUser": 50,
    "storage": {
        "host": "${AGAVE_SYSTEM_HOST}",
        "port": ${AGAVE_SYSTEM_PORT},
        "protocol": "SFTP",
        "rootDir": "/",
        "homeDir": "${AGAVE_STORAGE_HOME_DIR}",
        "auth": {
          "username" : "${MACHINE_USERNAME}",
          "publicKey" : "${PUB_KEY}",
          "privateKey" : "${PRIV_KEY}",
          "type" : "SSHKEYS"
        }
    },
    "workDir": "${AGAVE_STORAGE_WORK_DIR}"
}""")
!systems-addupdate -F ${AGAVE_EXECUTION_SYSTEM_ID}.txt

Writing file `sandbox-exec-dooley.txt'
Successfully added system sandbox-exec-dooley


## App registration

In [33]:
writefile("fork-app.txt","""
{  
   "name":"${AGAVE_USERNAME}-${MACHINE_NAME}-fork",
   "version":"1.0",
   "label":"Runs a command",
   "shortDescription":"Runs a command",
   "longDescription":"",
   "deploymentSystem":"${AGAVE_STORAGE_SYSTEM_ID}",
   "deploymentPath":"${AGAVE_APP_DEPLOYMENT_PATH}",
   "templatePath":"fork-wrapper.txt",
   "testPath":"fork-test.txt",
   "executionSystem":"${AGAVE_EXECUTION_SYSTEM_ID}",
   "executionType":"CLI",
   "parallelism":"SERIAL",
   "modules":[],
   "inputs":[
         {   
         "id":"datafile",
         "details":{  
            "label":"Data file",
            "description":"",
            "argument":null,
            "showArgument":false
         },
         "value":{  
            "default":"/dev/null",
            "order":0,
            "required":false,
            "validator":"",
            "visible":true
         }
      }   
   ],
   "parameters":[{
     "id" : "command",
     "value" : {
       "visible":true,
       "required":true,
       "type":"string",
       "order":0,
       "enquote":false,
       "default":"/bin/date",
       "validator":null
     },
     "details":{
         "label": "Command to run",
         "description": "This is the actual command you want to run. ex. df -h -d 1",
         "argument": null,
         "showArgument": false,
         "repeatArgument": false
     },
     "semantics":{
         "label": "Command to run",
         "description": "This is the actual command you want to run. ex. df -h -d 1",
         "argument": null,
         "showArgument": false,
         "repeatArgument": false
     }
   }],
   "outputs":[]
}
""")

writefile("fork-wrapper.txt","""
#!/bin/bash
\${command}
""")

writefile("fork-test.txt","""
command=date
fork-wrapper.txt
""")

Writing file `fork-app.txt'
Writing file `fork-wrapper.txt'
Writing file `fork-test.txt'


In [34]:
# upload files and create deployment path
!files-mkdir -S ${AGAVE_STORAGE_SYSTEM_ID} -N ${AGAVE_APP_DEPLOYMENT_PATH}
!files-upload -F fork-wrapper.txt -S ${AGAVE_STORAGE_SYSTEM_ID} ${AGAVE_APP_DEPLOYMENT_PATH}
!files-upload -F fork-test.txt -S ${AGAVE_STORAGE_SYSTEM_ID} ${AGAVE_APP_DEPLOYMENT_PATH}

# register app
!apps-addupdate -F fork-app.txt

Successfully created folder agave-deploy
Uploading fork-wrapper.txt...
######################################################################## 100.0%
Uploading fork-test.txt...
######################################################################## 100.0%
Successfully added app dooley-sandbox-fork-1.0


## Automation

In [35]:
%%writefile runagavecmd.py
from setvar import *

from time import sleep

def runagavecmd(cmd,infile=None):
    setvar("REMOTE_COMMAND="+cmd)
    setvar("REQUESTBIN_URL=$(requestbin-create)")
    print("")
    print(" ** QUERY STRING FOR REQUESTBIN **")
    print('%s?inspect'%os.environ['REQUESTBIN_URL'])
    print("")
    # The input file is an optional parameter, both
    # to our function and to the Agave application.
    if infile == None:
        setvar("INPUTS={}")
    else:
        setvar('INPUTS={"datafile":"'+infile+'"}')
    setvar("JOB_FILE=job-remote-$PID.txt")
    # Create the Json for the job file.
    writefile("$JOB_FILE","""
 {
   "name":"fork-command-1",
   "appId": "${AGAVE_USERNAME}-${MACHINE_NAME}-fork-1.0",
   "executionSystem": "${AGAVE_EXECUTION_SYSTEM_ID}",
   "archive": false,
   "notifications": [
    {
      "url":"${REQUESTBIN_URL}?event=\${EVENT}&jobid=\${JOB_ID}",
      "event":"*",
      "persistent":"true"
    }
   ],
   "parameters": {
     "command":"${REMOTE_COMMAND}"
   },
   "inputs":${INPUTS}
 }""")
    # Run the job and capture the output.
    setvar("""
# Capture the output of the job submit command
OUTPUT=$(jobs-submit -F $JOB_FILE)
# Parse out the job id from the output
JOB_ID=$(echo $OUTPUT | cut -d' ' -f4)
""")
    # Poll and wait for the job to finish.
    for iter in range(80): # Excessively generous
        setvar("STAT=$(jobs-status $JOB_ID)")
        stat = os.environ["STAT"]
        sleep(5.0)
        if stat == "FINISHED" or stat == "FAILED":
            break
    # Fetch the job output from the remote machine
    setvar("CMD=jobs-output-get ${JOB_ID} fork-command-1.out")
    os.system(os.environ["CMD"])
    print("All done! Output follows.")
    # Load the output into memory
    output=readfile("fork-command-1.out")
    print("=" * 70)
    print(output)

Writing runagavecmd.py


In [36]:
import runagavecmd as r
import imp
imp.reload(r)

<module 'runagavecmd' from '/home/jovyan/agave/runagavecmd.py'>

## Job inputs

In [40]:
writefile("input.txt","""
!INPUT FILE FOR FUNWAVE_TVD
  ! NOTE: all input parameter are capital sensitive
  ! --------------------TITLE-------------------------------------
  ! title only for log file
TITLE = VESSEL
  ! -------------------HOT START---------------------------------
HOT_START = F
FileNumber_HOTSTART = 1
  ! -------------------PARALLEL INFO-----------------------------
  ! 
  !    PX,PY - processor numbers in X and Y
  !    NOTE: make sure consistency with mpirun -np n (px*py)
  !    
PX = 2
PY = 1
  ! --------------------DEPTH-------------------------------------
  ! Depth types, DEPTH_TYPE=DATA: from depth file
  !              DEPTH_TYPE=FLAT: idealized flat, need depth_flat
  !              DEPTH_TYPE=SLOPE: idealized slope, 
  !                                 need slope,SLP starting point, Xslp
  !                                 and depth_flat
DEPTH_TYPE = FLAT
DEPTH_FLAT = 10.0
  ! -------------------PRINT---------------------------------
  ! PRINT*,
  ! result folder
RESULT_FOLDER = output/

  ! ------------------DIMENSION-----------------------------
  ! global grid dimension
Mglob = 500
Nglob = 100

  ! ----------------- TIME----------------------------------
  ! time: total computational time/ plot time / screen interval 
  ! all in seconds
TOTAL_TIME = 3.0
PLOT_INTV = 1.0
PLOT_INTV_STATION = 50000.0
SCREEN_INTV = 1.0
HOTSTART_INTV = 360000000000.0

WAVEMAKER = INI_GAU
AMP = 3.0
Xc = 250.0
Yc = 50.0
WID = 20.0

  ! -----------------GRID----------------------------------
  ! if use spherical grid, in decimal degrees
  ! cartesian grid sizes
DX = 1.0
DY = 1.0
  ! ----------------SHIP WAKES ----------------------------
VESSEL_FOLDER = ./
NumVessel = 2
  ! -----------------OUTPUT-----------------------------
ETA = T
U = T
V = T
""")

writefile("run.sh","""
#!/bin/bash
export LD_LIBRARY_PATH=/usr/local/lib
mkdir -p rundir
cd ./rundir
cp ../input.txt .
mpirun -np 2 ~/FUNWAVE-TVD/src/funwave_vessel
""")
!files-upload -F input.txt -S ${AGAVE_STORAGE_SYSTEM_ID} .
!files-upload -F run.sh -S ${AGAVE_STORAGE_SYSTEM_ID} .


Writing file `input.txt'
Writing file `run.sh'
Uploading input.txt...
######################################################################## 100.0%
Uploading run.sh...
######################################################################## 100.0%


In [38]:
files-upload -F fork-wrapper.txt -S ${AGAVE_STORAGE_SYSTEM_ID} ${AGAVE_APP_DEPLOYMENT_PATH}


SyntaxError: invalid syntax (<ipython-input-38-93778f6ac1f7>, line 1)